In [77]:
# 1) Design model (input size, output size, forward pass)
# 2) Construct loss and optimizer
# 3) Training loop
#    - forward pass: compute prediction and loss
#    - backward pass: gradients
#    - update weights


# MNIST
# DataLoader, Transformation
# Multilayer Neural Net, activation function
# Loss and Optimizer
# Training Loop (batch training)
# Model evaluation
# GPU support

# ENV: Python 3.9.12

In [78]:
# pip install tensorboard
# terminal: 
# $cd folder
# $tensorboard --logdir=runs
# brower: http://localhost:6006/

# Tasks:
# 1. show images at TensorBoard
# 2. show computaional graph
# 3. show training loss/acc with different learning rates
#    - change the learning rate: 0.001 --> 0.01
#    - change the folder: mnist --> mnist001
#    - run the training again
#    - refresh tensorboard; you will see two curves.
# 4. show precision recall curve

# https://pytorch.org/docs/stable/tensorboard.html#

In [79]:
import torch
import torch.nn as nn
import torch.nn.functional as fnc
import torchvision # some builtin datasets
import torchvision.transforms as transforms

from torch.utils.data import Dataset, DataLoader

import math
import matplotlib.pyplot as plt
import sys

from torch.utils.tensorboard import SummaryWriter

from PIL import Image as pil
pil.ANTIALIAS=pil.LANCZOS

In [80]:
# writer = SummaryWriter("./runs/mnist")
writer = SummaryWriter("./runs/mnist001") # learning rate 0.01

In [81]:
# Settings

# device config
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

# hyper parameters
input_size = 28*28
hidden_size = 100
num_classes = 10
num_epoches = 2
batch_size = 100
learning_rate = 0.01

device: cuda


In [82]:
# MNIST
train_data = torchvision.datasets.MNIST(
    root="./data", 
    train=True,
    transform=transforms.ToTensor(),#Converts the images into PyTorch tensors. Each image is normalized to have values in the range [0,1]
    download=True)

test_data = torchvision.datasets.MNIST(
    root="./data", 
    train=False,
    transform=transforms.ToTensor())

In [83]:
train_loader = torch.utils.data.DataLoader(
    dataset=train_data, 
    batch_size=batch_size,
    shuffle=True)

test_loader = torch.utils.data.DataLoader(
    dataset=test_data, 
    batch_size=batch_size,
    shuffle=True)

examples = iter(train_loader)
samples, labels = examples.next()
print(samples.shape, labels.shape)
# [100, 1, 28, 28]: batch size 100, gray image with size 28 x28

#examples = iter(test_loader)
#samples, labels = examples.next()
#print(samples.shape, labels.shape)

# for i in range(6):
#    plt.subplot(2, 3, i+1)
#    plt.imshow(samples[i][0], cmap='gray')
# plt.show()

# Task 1: show images
img_grid = torchvision.utils.make_grid(samples)
writer.add_image("mnist_images", img_grid)
writer.close()

torch.Size([100, 1, 28, 28]) torch.Size([100])


In [84]:
# Model
class MultiCLSNeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MultiCLSNeuralNet, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.linear1(x) # input: [batch size, 28x28], output: [batch size, 100]
        out = self.relu(out)
        out = self.linear2(out) # output: [batch size, 10]
        return out

model = MultiCLSNeuralNet(input_size=input_size, hidden_size=hidden_size, num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Task 2: show computational graph
writer.add_graph(model, samples.to(device).reshape(-1, 28*28))
writer.close()

In [85]:
# Training loop
num_total_steps = len(train_loader)

running_loss = 0.0
running_correct = 0
for epoch in range(num_epoches):
    for i, (images, labels) in enumerate(train_loader):
        # print(images.shape) # torch.Size([100, 1, 28, 28]) --> 100, 784
        # print(labels.shape) # torch.Size([100])
        
        images = images.view(batch_size, -1).to(device)
        labels = labels.to(device)
        
        # forward
        y_pred = model(images)
        loss = criterion(y_pred, labels)
        _, preds = torch.max(y_pred, 1)
        
        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # track training performance
        running_loss += loss.item()
        running_correct += (preds == labels).sum().item()
        if (i+1) % 100 == 0: # average loss or acc in 100 steps
            writer.add_scalar('training loss', running_loss/100, epoch*num_total_steps + i)
            writer.add_scalar('training acc', running_correct/100, epoch*num_total_steps + i)
            running_loss = 0.0
            running_correct = 0

        if (i+1) % 100 == 0:
            print(f'epoch {epoch+1} / {num_epoches}, step {i+1} / {num_total_steps}, loss = {loss: 0.4f}')

writer.close()

epoch 1 / 2, step 100 / 600, loss =  0.3075
epoch 1 / 2, step 200 / 600, loss =  0.1161
epoch 1 / 2, step 300 / 600, loss =  0.2285
epoch 1 / 2, step 400 / 600, loss =  0.1460
epoch 1 / 2, step 500 / 600, loss =  0.1146
epoch 1 / 2, step 600 / 600, loss =  0.2495
epoch 2 / 2, step 100 / 600, loss =  0.1456
epoch 2 / 2, step 200 / 600, loss =  0.0313
epoch 2 / 2, step 300 / 600, loss =  0.0787
epoch 2 / 2, step 400 / 600, loss =  0.1671
epoch 2 / 2, step 500 / 600, loss =  0.1406
epoch 2 / 2, step 600 / 600, loss =  0.0639


In [86]:
# Evaluation

GT_labels = []
pred_probs = []
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        images = images.view(batch_size, -1).to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        _, y_cls = torch.max(outputs, 1)
        probs = fnc.softmax(outputs, dim=1)
        
        GT_labels.append(labels)
        pred_probs.append(probs)

        n_samples += labels.shape[0]
        n_correct += (y_cls == labels).sum().item()
    
    acc = 100*n_correct/n_samples
    print(f'acc = {acc:.4f}')

acc = 97.0100


In [87]:
GT_labels1 = torch.cat(GT_labels).cpu().numpy()
print(GT_labels1.shape)

pred_probs1 = torch.cat(pred_probs).cpu().numpy()
print(pred_probs1.shape)

for class_idx in range(pred_probs1.shape[1]):
    writer.add_pr_curve(
        f'Class {class_idx} Precision Recall',
        GT_labels1 == class_idx,
        pred_probs1[:, class_idx],
        global_step = 0
    )
writer.close()

(10000,)
(10000, 10)
